# 0.5 Notebook de Treinamento (Execução no Google Colab)

Este notebook tem como objetivo **exclusivamente o treinamento dos modelos** utilizados no projeto.

Ele foi estruturado para execução no **Google Colab com GPU**, visando:

- Acelerar o treinamento (uso de GPU T4);
- Garantir estabilidade do ambiente;
- Evitar problemas de compatibilidade local.

## ⚠️ Observação Importante sobre Estrutura

Este notebook contém a **repetição intencional de algumas funções** que também estão implementadas no diretório `src/` do repositório.

Essa duplicação ocorre porque:

- O ambiente do Colab apresentou dificuldades na importação direta dos módulos do `src/`;
- O objetivo principal aqui é permitir o treinamento isolado e reprodutível;
- O pipeline modular oficial (engenharia de software organizada) está no diretório `src/`.

Portanto:

- 📁 `src/` → implementação modular (inference, model, transforms, etc.)
- 📓 `train.ipynb` → notebook dedicado apenas ao treinamento e geração dos `.pth`

## 🧠 Fluxo do Projeto

1. Este notebook treina os modelos e salva os pesos (`.pth`).
2. O notebook principal de competição utiliza os pesos salvos.
3. A estrutura modular no `src/` garante organização e reprodutibilidade.

Essa separação melhora:

- Organização do repositório
- Clareza metodológica
- Reprodutibilidade experimental
- Avaliação de engenharia de software

In [1]:
# 0.5.1 Montar Google Drive

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## 0.5.2 Definir o caminho do dataset (DATA_DIR)

Defina abaixo a pasta no Drive onde você colocou o dataset extraído.
Recomendação: manter o notebook e a pasta do dataset no mesmo diretório (organização do projeto).


In [2]:
# 0.5.2 Defina o caminho base do seu projeto no Drive (ajuste só essa linha)

PROJECT_DIR = "/content/drive/MyDrive/ligia-cv"  # <- ajuste para sua pasta
DATA_DIR = f"{PROJECT_DIR}/ligia-compviz"            # <- pasta do dataset extraído

print("PROJECT_DIR:", PROJECT_DIR)
print("DATA_DIR:", DATA_DIR)


PROJECT_DIR: /content/drive/MyDrive/ligia-cv
DATA_DIR: /content/drive/MyDrive/ligia-cv/ligia-compviz


## 0.5.3 Sanity Check (estrutura do dataset)

Validamos se os arquivos/pastas essenciais existem antes de seguir.
Isso evita erros comuns como:
- dataset na pasta errada
- dataset ainda zipado
- estrutura diferente do esperado

In [3]:
# 0.5.3 Sanity check enxuto

import os

expected = {
    "train.csv": os.path.join(DATA_DIR, "train.csv"),
    "test.csv": os.path.join(DATA_DIR, "test.csv"),
    "NORMAL dir": os.path.join(DATA_DIR, "train", "train", "NORMAL"),
    "PNEUMONIA dir": os.path.join(DATA_DIR, "train", "train", "PNEUMONIA"),
    "test_images dir": os.path.join(DATA_DIR, "test_images", "test_images"),
}

for name, path in expected.items():
    assert os.path.exists(path), f"❌ Não encontrado: {name} -> {path}"
print("✅ Estrutura mínima OK.")

# Contagens rápidas
n_norm = len(os.listdir(expected["NORMAL dir"]))
n_pne  = len(os.listdir(expected["PNEUMONIA dir"]))
n_test = len(os.listdir(expected["test_images dir"]))

print(f"📊 Contagens: NORMAL={n_norm} | PNEUMONIA={n_pne} | TEST={n_test}")
print("✅ DATA_DIR pronto:", DATA_DIR)


✅ Estrutura mínima OK.
📊 Contagens: NORMAL=1349 | PNEUMONIA=3883 | TEST=624
✅ DATA_DIR pronto: /content/drive/MyDrive/ligia-cv/ligia-compviz


# 1. Setup Experimental

Nesta seção realizamos a configuração inicial do ambiente experimental.

O objetivo é:

- Garantir **reprodutibilidade científica**
- Importar bibliotecas necessárias
- Definir o dispositivo de execução (CPU/GPU)
- Estruturar os caminhos do dataset

Seguindo as diretrizes do edital, fixamos a semente aleatória (seed = 42) para assegurar consistência nos resultados ao longo das execuções.

Essa etapa é essencial para garantir:
- Controle experimental
- Robustez metodológica
- Integridade científica


In [4]:
# Bibliotecas Fundamentais
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

# Deep Learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# Métricas
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score


## 1.1 Reprodutibilidade

Para garantir a consistência dos experimentos, fixamos a semente aleatória em todas as bibliotecas relevantes.

Isso evita variações causadas por:

- Inicialização aleatória dos pesos
- Embaralhamento de dados
- Operações internas do backend CUDA

Essa prática é essencial em experimentos científicos e será mantida ao longo de todo o projeto.


In [5]:
# Reprodutibilidade
SEED = 42

def seed_everything(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)


## 1.2 Configuração do Dispositivo

O treinamento será realizado utilizando GPU quando disponível,
de forma a acelerar o processo de otimização do modelo.

Caso GPU não esteja disponível, o código executará automaticamente em CPU.


In [6]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Executando em: {device}")


Executando em: cuda


## 1.3 Estrutura do Dataset

O dataset está organizado da seguinte forma:

- `test_images/test_images/` → imagens do conjunto de teste (sem rótulos)
- `train/train/NORMAL/` → imagens normais
- `train/train/PNEUMONIA/` → imagens com pneumonia
- `train/train.csv` → arquivo auxiliar
- `train/test.csv` → template de submissão

As imagens de treino estão separadas por classe em pastas distintas,
permitindo a construção manual de um DataFrame estruturado.


In [7]:
# Paths do Dataset

TEST_IMG_DIR  = os.path.join(DATA_DIR, "test_images/test_images")
TRAIN_ROOT    = os.path.join(DATA_DIR, "train")
TRAIN_IMG_DIR = os.path.join(TRAIN_ROOT, "train")

TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
TEST_CSV  = os.path.join(DATA_DIR, "test.csv")

print("Test images:", TEST_IMG_DIR)
print("Train images:", TRAIN_IMG_DIR)


Test images: /content/drive/MyDrive/ligia-cv/ligia-compviz/test_images/test_images
Train images: /content/drive/MyDrive/ligia-cv/ligia-compviz/train/train


## 1.4 Verificação dos Caminhos (Sanity Check)

Antes de iniciar qualquer análise ou treinamento, validamos se os diretórios esperados realmente existem e se contêm arquivos.

Essa etapa evita erros comuns, como:
- caminho incorreto no `/kaggle/input/`
- pastas vazias por configuração errada do dataset
- divergências na estrutura de diretórios


In [8]:
print("TEST_IMG_DIR existe?", os.path.exists(TEST_IMG_DIR))
print("TRAIN_IMG_DIR existe?", os.path.exists(TRAIN_IMG_DIR))

print("\nAmostra de arquivos em test_images:", os.listdir(TEST_IMG_DIR)[:5])
print("Subpastas dentro de train/train:", os.listdir(TRAIN_IMG_DIR))

# contagem rápida
print("\nQtd test_images:", len(os.listdir(TEST_IMG_DIR)))
print("Qtd pneumonia:", len(os.listdir(os.path.join(TRAIN_IMG_DIR, "PNEUMONIA"))))
print("Qtd normal:", len(os.listdir(os.path.join(TRAIN_IMG_DIR, "NORMAL"))))


TEST_IMG_DIR existe? True
TRAIN_IMG_DIR existe? True

Amostra de arquivos em test_images: ['img_0028.jpeg', 'img_0009.jpeg', 'img_0033.jpeg', 'img_0006.jpeg', 'img_0032.jpeg']
Subpastas dentro de train/train: ['PNEUMONIA', 'NORMAL']

Qtd test_images: 624
Qtd pneumonia: 3883
Qtd normal: 1349


## 3.1 Estrutura do Conjunto de Treino

O conjunto de treino está organizado em subpastas por classe, o que facilita a inspeção e a criação de um DataFrame para treinamento posteriormente.

- `PNEUMONIA/` → classe positiva (1)
- `NORMAL/` → classe negativa (0)

Nesta etapa, definimos os caminhos dessas pastas para facilitar a análise exploratória e garantir consistência ao longo do notebook.


In [9]:
PNE_DIR = os.path.join(TRAIN_IMG_DIR, "PNEUMONIA")
NOR_DIR = os.path.join(TRAIN_IMG_DIR, "NORMAL")

assert os.path.exists(PNE_DIR), "Pasta PNEUMONIA não encontrada."
assert os.path.exists(NOR_DIR), "Pasta NORMAL não encontrada."

print("PNE_DIR:", PNE_DIR)
print("NOR_DIR:", NOR_DIR)


PNE_DIR: /content/drive/MyDrive/ligia-cv/ligia-compviz/train/train/PNEUMONIA
NOR_DIR: /content/drive/MyDrive/ligia-cv/ligia-compviz/train/train/NORMAL


# 4. Construção do Dataset e Estratégia de Validação

Nesta seção construímos os DataFrames de treino e teste de forma consistente com os arquivos oficiais da competição:

- `train.csv` contém a referência oficial (`id`, `label`) para treinamento.
- `test.csv` contém os `id`s esperados no arquivo de submissão.

Além disso:
- validamos se os arquivos existentes nas pastas correspondem aos `id`s dos CSVs;
- montamos o caminho completo de cada imagem (`path`);
- definimos uma estratégia de validação com **StratifiedKFold**, preservando o desbalanceamento de classes em cada fold.


## 4.1 Carregamento dos CSVs Oficiais

Os CSVs fornecidos pela competição são a fonte oficial de:

- `id`: identificador do arquivo (inclui extensão `.jpeg`)
- `label`: rótulo binário no conjunto de treino

Nesta etapa, carregamos os CSVs e inspecionamos seu formato.


In [10]:
# 4.1 Leitura dos CSVs

train_csv = pd.read_csv(TRAIN_CSV)
test_csv  = pd.read_csv(TEST_CSV)

display(train_csv.head())
display(test_csv.head())

print("Colunas train.csv:", train_csv.columns.tolist())
print("Colunas test.csv:", test_csv.columns.tolist())
print("Shapes:", train_csv.shape, test_csv.shape)


,id,label
0,NORMAL-8648239-0004.jpeg,0
1,NORMAL-388586-0003.jpeg,0
2,NORMAL-7824011-0001.jpeg,0
3,NORMAL-8234246-0001.jpeg,0
4,NORMAL-5505017-0001.jpeg,0


,id
0,img_0001.jpeg
1,img_0002.jpeg
2,img_0003.jpeg
3,img_0004.jpeg
4,img_0005.jpeg


Colunas train.csv: ['id', 'label']
Colunas test.csv: ['id']
Shapes: (5232, 2) (624, 1)


## 4.2 Construção do DataFrame de Treino (`df_train`)

Construímos `df_train` a partir do `train.csv`, garantindo que:

- cada `id` tenha um arquivo correspondente na pasta de imagens;
- o `label` utilizado é o rótulo oficial fornecido pela competição.

Também criamos a coluna `path` com o caminho completo da imagem para uso no DataLoader.


In [11]:
# 4.2 df_train: CSV -> path

# Conjunto de arquivos existentes nas pastas (ids com extensão)
files_pne = set(os.listdir(PNE_DIR))
files_nor = set(os.listdir(NOR_DIR))
files_all = files_pne | files_nor

# Checagem: todo id do train.csv precisa existir nas pastas
missing_train_files = set(train_csv["id"]) - files_all
print("Arquivos do train.csv que NÃO estão nas pastas:", len(missing_train_files))
assert len(missing_train_files) == 0, "Existem ids no train.csv sem arquivo correspondente nas pastas."

# Função para criar o path correto de cada id
def build_train_path(img_id: str) -> str:
    # Decide se o arquivo está em PNE_DIR ou NOR_DIR
    if img_id in files_pne:
        return os.path.join(PNE_DIR, img_id)
    elif img_id in files_nor:
        return os.path.join(NOR_DIR, img_id)
    else:
        # Isso não deveria acontecer por causa do assert acima
        return None

df_train = train_csv.copy()
df_train["path"] = df_train["id"].apply(build_train_path)

# Checagem final: nenhum path pode ficar nulo
assert df_train["path"].isna().sum() == 0, "Alguns paths ficaram nulos. Verifique as pastas."

# Embaralha para não ficar com blocos ordenados
df_train = df_train.sample(frac=1, random_state=SEED).reset_index(drop=True)

print("df_train shape:", df_train.shape)
df_train.head()


Arquivos do train.csv que NÃO estão nas pastas: 0
df_train shape: (5232, 3)


,id,label,path
0,NORMAL-9543520-0001.jpeg,0,/content/drive/MyDrive/ligia-cv/ligia-compviz/...
1,NORMAL-8714707-0001.jpeg,0,/content/drive/MyDrive/ligia-cv/ligia-compviz/...
2,VIRUS-2750119-0003.jpeg,1,/content/drive/MyDrive/ligia-cv/ligia-compviz/...
3,BACTERIA-1157929-0001.jpeg,1,/content/drive/MyDrive/ligia-cv/ligia-compviz/...
4,BACTERIA-6171093-0001.jpeg,1,/content/drive/MyDrive/ligia-cv/ligia-compviz/...


## 4.3 Construção do DataFrame de Teste (`df_test`)

No conjunto de teste, os rótulos não são fornecidos. O objetivo é:

- associar cada `id` do `test.csv` ao respectivo arquivo na pasta `test_images`;
- garantir que os `ids` utilizados na submissão sigam exatamente o formato esperado (incluindo `.jpeg`).


In [12]:
# 4.3 df_test: test.csv -> path

# Arquivos disponíveis na pasta de teste
test_files_in_dir = set(os.listdir(TEST_IMG_DIR))

# Checagem: todo id do test.csv precisa existir na pasta
missing_test_files = set(test_csv["id"]) - test_files_in_dir
print("Arquivos do test.csv que NÃO estão na pasta:", len(missing_test_files))
assert len(missing_test_files) == 0, "Existem ids no test.csv sem arquivo correspondente em test_images."

df_test = test_csv.copy()  # mantém o id oficial
df_test["path"] = df_test["id"].apply(lambda x: os.path.join(TEST_IMG_DIR, x))

print("df_test shape:", df_test.shape)
df_test.head()


Arquivos do test.csv que NÃO estão na pasta: 0
df_test shape: (624, 2)


,id,path
0,img_0001.jpeg,/content/drive/MyDrive/ligia-cv/ligia-compviz/...
1,img_0002.jpeg,/content/drive/MyDrive/ligia-cv/ligia-compviz/...
2,img_0003.jpeg,/content/drive/MyDrive/ligia-cv/ligia-compviz/...
3,img_0004.jpeg,/content/drive/MyDrive/ligia-cv/ligia-compviz/...
4,img_0005.jpeg,/content/drive/MyDrive/ligia-cv/ligia-compviz/...


## 4.4 Validação com StratifiedKFold

Como o dataset é desbalanceado, utilizamos **StratifiedKFold** para que cada fold mantenha uma proporção de classes semelhante ao conjunto original.

Isso melhora a confiabilidade da validação e reduz variações artificiais entre folds.


In [13]:
# 4.4 Criação dos folds

N_SPLITS = 5

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

df_train["fold"] = -1  # coluna para guardar o fold de cada amostra

for fold, (_, val_idx) in enumerate(skf.split(df_train["path"], df_train["label"])):
    df_train.loc[val_idx, "fold"] = fold

# Checagem: distribuição de classes por fold (deve ser semelhante)
for f in range(N_SPLITS):
    counts = df_train[df_train["fold"] == f]["label"].value_counts(normalize=True)
    print(f"Fold {f} proporções:\n{counts}\n")

df_train.head()


Fold 0 proporções:
label
1    0.74212
0    0.25788
Name: proportion, dtype: float64

Fold 1 proporções:
label
1    0.74212
0    0.25788
Name: proportion, dtype: float64

Fold 2 proporções:
label
1    0.741874
0    0.258126
Name: proportion, dtype: float64

Fold 3 proporções:
label
1    0.741874
0    0.258126
Name: proportion, dtype: float64

Fold 4 proporções:
label
1    0.74283
0    0.25717
Name: proportion, dtype: float64



,id,label,path,fold
0,NORMAL-9543520-0001.jpeg,0,/content/drive/MyDrive/ligia-cv/ligia-compviz/...,1
1,NORMAL-8714707-0001.jpeg,0,/content/drive/MyDrive/ligia-cv/ligia-compviz/...,3
2,VIRUS-2750119-0003.jpeg,1,/content/drive/MyDrive/ligia-cv/ligia-compviz/...,3
3,BACTERIA-1157929-0001.jpeg,1,/content/drive/MyDrive/ligia-cv/ligia-compviz/...,3
4,BACTERIA-6171093-0001.jpeg,1,/content/drive/MyDrive/ligia-cv/ligia-compviz/...,3


# 5. Pré-processamento e DataLoaders

Nesta seção definimos como as imagens serão preparadas antes de entrarem no modelo.

Inclui:

- Definição do tamanho padrão da imagem (`IMG_SIZE`)
- Transformações para treino (com data augmentation controlado)
- Transformações para validação/teste (sem augmentation)
- Criação de um `Dataset` customizado e `DataLoader`

Como utilizaremos modelos pré-treinados, aplicamos normalização compatível com ImageNet.
Além disso, aplicamos augmentation apenas no treino para melhorar generalização,
mantendo validação/teste determinísticos.


## 5.1 Transformações (Transforms)

Como vimos no EDA, as imagens possuem resoluções e aspect ratios variados.
Para alimentar redes neurais, padronizamos o tamanho.

- **Treino:** inclui augmentations leves (ex.: rotação pequena e flip horizontal),
  visando robustez a variações de aquisição.
- **Validação/Teste:** apenas resize + normalização, para medir desempenho de forma estável.


In [14]:
IMG_SIZE = 224  # baseline comum em modelos pré-treinados (ResNet/EfficientNet)

# Transformações do treino (com augmentation leve)
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),        # padroniza o tamanho
    transforms.RandomHorizontalFlip(p=0.5),          # variação leve (não altera anatomia verticalmente)
    transforms.RandomRotation(degrees=10),           # rotação pequena (simula variação de posicionamento)
    transforms.ToTensor(),                          # converte para tensor [0,1]
    transforms.Normalize([0.485, 0.456, 0.406],      # normalização ImageNet
                         [0.229, 0.224, 0.225]),
])

# Transformações para validação e teste (sem augmentation)
valid_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])


## 5.2 Dataset Customizado

Criamos um Dataset customizado para:

- Ler imagens a partir da coluna `path`
- Retornar `(imagem, label)` no treino/validação
- Retornar apenas `imagem` no teste

A conversão para `RGB` é utilizada para compatibilidade direta com modelos pré-treinados em ImageNet.


In [15]:
class XRayDataset(Dataset):
    def __init__(self, df, transform=None, has_label=True):

        # df: DataFrame com colunas:
        # - path (obrigatório)
        # - label (se has_label=True)

        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.has_label = has_label

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Abre imagem e converte para RGB (compatível com modelos ImageNet)
        img = Image.open(row["path"]).convert("RGB")

        # Aplica transformações
        if self.transform:
            img = self.transform(img)

        # Se houver label (treino/validação), retorna (img, label)
        if self.has_label:
            y = torch.tensor(row["label"], dtype=torch.float32)
            return img, y

        # Caso teste, retorna apenas imagem
        return img


## 5.3 DataLoaders com Separação por Fold

Usaremos a coluna `fold` criada na Seção 4 para separar treino e validação.

Isso garante que:
- treino e validação são disjuntos
- a validação preserva a distribuição de classes (stratified)


In [16]:
def make_loaders(df_train, fold=0, batch_size=32, num_workers=2):
    # Separa treino e validação
    df_tr = df_train[df_train["fold"] != fold].reset_index(drop=True)
    df_va = df_train[df_train["fold"] == fold].reset_index(drop=True)

    # Cria datasets
    ds_tr = XRayDataset(df_tr, transform=train_tfms, has_label=True)
    ds_va = XRayDataset(df_va, transform=valid_tfms, has_label=True)

    # DataLoaders
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, shuffle=True,
                       num_workers=num_workers, pin_memory=True)
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False,
                       num_workers=num_workers, pin_memory=True)

    return dl_tr, dl_va

# Exemplo: loaders do fold 0
train_loader, valid_loader = make_loaders(df_train, fold=0, batch_size=32)

# Test loader
test_dataset = XRayDataset(df_test, transform=valid_tfms, has_label=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

print("Train batches:", len(train_loader))
print("Valid batches:", len(valid_loader))
print("Test batches :", len(test_loader))


Train batches: 131
Valid batches: 33
Test batches : 20


# 6. Modelagem e Treinamento

Nesta seção definimos e treinamos um modelo de Deep Learning para classificar
imagens de raio-X torácico em:

- 0 → Normal
- 1 → Pneumonia

Como estratégia principal, utilizamos **Transfer Learning** com uma rede pré-treinada em ImageNet.
Isso é útil pois o dataset não é gigantesco e o modelo pré-treinado já possui filtros visuais
úteis (bordas, texturas, padrões), acelerando a convergência.

A saída do modelo será um **score contínuo** (logit), que será convertido em probabilidade via sigmoide.
A métrica de validação será a **ROC-AUC**, compatível com a avaliação da competição.


## 6.1 Arquitetura do Modelo

Usaremos a EfficientNet-B0 pré-treinada. Substituímos a camada final para produzir 1 logit:

- logit → `sigmoid(logit)` = probabilidade de pneumonia

Essa configuração é adequada para classificação binária com `BCEWithLogitsLoss`,
que aplica internamente a sigmoide de forma numericamente estável.


In [17]:
import torchvision.models as models
import torch.nn as nn

def build_model():
    # Carrega EfficientNet-B0 pré-treinada
    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

    # Troca a camada final para saída binária (1 logit)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, 1)

    return model


## 6.2 Função de Perda e Otimizador

Como o dataset é desbalanceado (maioria pneumonia), utilizamos `pos_weight` na loss
para penalizar mais erros na classe positiva (pneumonia).

A loss escolhida é:

- `BCEWithLogitsLoss(pos_weight=...)`

Otimizador:
- `AdamW` (boa estabilidade e regularização via weight decay)

Scheduler:
- `CosineAnnealingLR` (reduz a taxa de aprendizado suavemente ao longo das épocas)


In [18]:
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

def get_criterion(df):
    # Calcula pos_weight = (negativos / positivos)
    pos = (df["label"] == 1).sum()
    neg = (df["label"] == 0).sum()
    pos_weight = torch.tensor([neg / pos], device=device, dtype=torch.float32)

    print("pos_weight:", pos_weight.item())
    return nn.BCEWithLogitsLoss(pos_weight=pos_weight)

criterion = get_criterion(df_train)


pos_weight: 0.34741178154945374


## 6.3 Treino e Validação

Durante o treino, o modelo recebe imagens e retorna logits.
Na validação, convertemos logits em probabilidades usando `sigmoid` e calculamos ROC-AUC.

Importante:
- ROC-AUC usa **scores contínuos** (probabilidades), não rótulos 0/1.


In [19]:
from sklearn.metrics import roc_auc_score
import numpy as np

def train_one_epoch(model, loader, optimizer):
    model.train()
    running_loss = 0.0

    for x, y in loader:
        x = x.to(device)
        y = y.to(device).unsqueeze(1)  # shape (B,1)

        optimizer.zero_grad()

        logits = model(x)             # saída: logit
        loss = criterion(logits, y)   # BCEWithLogitsLoss

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)

    return running_loss / len(loader.dataset)

@torch.no_grad()
def valid_one_epoch(model, loader):
    model.eval()
    probs = []
    ys = []

    for x, y in loader:
        x = x.to(device)

        logits = model(x)
        p = torch.sigmoid(logits).cpu().numpy().ravel()  # probabilidade de pneumonia

        probs.append(p)
        ys.append(y.numpy().ravel())

    probs = np.concatenate(probs)
    ys = np.concatenate(ys)

    auc = roc_auc_score(ys, probs)
    return auc

## 6.4 Treinamento Inicial (Fold único)

Primeiro treinamos em um único fold para validar o pipeline completo:

- DataLoader
- Modelo
- Loss
- AUC

Após confirmar que está funcionando, expandiremos para 5 folds e ensemble.


In [ ]:
# Configuração inicial (Fold único) + salvar best_state

FOLD = 0
EPOCHS = 3
LR = 3e-4
BATCH_SIZE = 32

train_loader, valid_loader = make_loaders(df_train, fold=FOLD, batch_size=BATCH_SIZE)

model = build_model().to(device)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

best_auc = -1
best_state = None  # garante que existe mesmo se algo der errado

for epoch in range(1, EPOCHS + 1):
    train_loss = train_one_epoch(model, train_loader, optimizer)
    val_auc = valid_one_epoch(model, valid_loader)
    scheduler.step()

    print(f"Epoch {epoch}/{EPOCHS} | loss={train_loss:.4f} | val_auc={val_auc:.4f}")

    # Salva o melhor checkpoint (na CPU para economizar VRAM e facilitar reutilização)
    if val_auc > best_auc:
        best_auc = val_auc
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

print("Best AUC:", best_auc)

# Recarrega o melhor estado no modelo (para garantir que o 'model' final é o melhor)
model.load_state_dict(best_state)
model = model.to(device)
print("✅ Best model carregado no objeto 'model'.")

# (Opcional) Salvar em arquivo para usar depois (Seção 7 / Grad-CAM) sem retreinar
torch.save(best_state, f"best_model_fold{FOLD}.pth")
print(f"✅ Checkpoint salvo em: best_model_fold{FOLD}.pth")


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 234MB/s]


## 6.5 Treinamento com Validação Cruzada (5-Fold) e Ensemble

Para aumentar robustez e reduzir variância do modelo, treinamos 1 modelo por fold.

Para o conjunto de teste, geramos probabilidades com cada modelo e tiramos a média (ensemble),
o que normalmente melhora a performance em competições Kaggle.

Nesta etapa registramos:
- AUC de cada fold
- AUC médio
- Probabilidades finais no teste (média dos folds)


In [ ]:
@torch.no_grad()
def predict_proba(model, loader):
    #Retorna probabilidades (sigmoid) para todos os itens do loader, na ordem.
    model.eval()
    all_probs = []
    for batch in loader:
        # batch pode ser (x,y) ou só x (teste)
        if isinstance(batch, (list, tuple)) and len(batch) == 2:
            x, _ = batch
        else:
            x = batch

        x = x.to(device)
        logits = model(x)
        probs = torch.sigmoid(logits).detach().cpu().numpy().ravel()
        all_probs.append(probs)

    return np.concatenate(all_probs)


In [ ]:
N_SPLITS = 5
EPOCHS = 3
LR = 3e-4
BATCH_SIZE = 32
NUM_WORKERS = 2

fold_aucs = []
test_pred_folds = []

best_states = []        # guarda best_state em memória (opcional, mas útil)
val_probs_folds = []    # (opcional) probs de validação por fold
val_true_folds = []     # (opcional) labels verdadeiros por fold

# dataset/loader de teste
test_dataset = XRayDataset(df_test, transform=valid_tfms, has_label=False)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=(device.type == "cuda")
)

for fold in range(N_SPLITS):
    print(f"\n==================== FOLD {fold} ====================")

    # Loaders do fold atual
    train_loader, valid_loader = make_loaders(
        df_train, fold=fold, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS
    )

    # Modelo + otimizador + scheduler
    model = build_model().to(device)
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    best_auc = -1
    best_state = None

    for epoch in range(1, EPOCHS + 1):
        train_loss = train_one_epoch(model, train_loader, optimizer)
        val_auc = valid_one_epoch(model, valid_loader)
        scheduler.step()

        print(f"Epoch {epoch}/{EPOCHS} | loss={train_loss:.4f} | val_auc={val_auc:.4f}")

        if val_auc > best_auc:
            best_auc = val_auc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

    # Guarda AUC do fold
    fold_aucs.append(best_auc)
    print(f"Best AUC (fold {fold}): {best_auc:.6f}")

    # Salva best_state em memória (opcional)
    best_states.append(best_state)

    # Salva best_state em arquivo (recomendado)
    ckpt_path = f"best_model_fold{fold}.pth"
    torch.save(best_state, ckpt_path)
    print(f"✅ Checkpoint salvo em: {ckpt_path}")

    # Carrega melhor estado e prediz no teste
    model.load_state_dict(best_state)
    model = model.to(device)

    test_probs = predict_proba(model, test_loader)
    test_pred_folds.append(test_probs)

    # (Opcional) salvar probs da validação (para Seção 7 sem retrain)
    df_va = df_train[df_train["fold"] == fold].reset_index(drop=True)
    val_probs = predict_proba(model, valid_loader)  # valid_loader já está no fold certo e sem shuffle
    val_probs_folds.append(val_probs)
    val_true_folds.append(df_va["label"].values)

# Resultado final
print("\nAUCs por fold:", [round(float(a), 6) for a in fold_aucs])
print("AUC médio:", float(np.mean(fold_aucs)))

# Ensemble (média das probabilidades dos folds)
test_pred_mean = np.mean(np.stack(test_pred_folds, axis=0), axis=0)
print("test_pred_mean shape:", test_pred_mean.shape)
